In [1]:
import requests
from html.parser import HTMLParser
from urllib.parse import urlparse, parse_qs


class GetForm(HTMLParser):
    def handle_starttag(self, tag, attrs):
        if tag == "form":
            self.tag, self.attrs = tag, dict(attrs)
        return super().handle_starttag(tag, attrs)

def get_token(response):
    gd_parser = GetForm()
    gd_parser.feed(response.text)
    params = parse_qs(urlparse(gd_parser.attrs["action"]).query)
    return params["uuid"][0]

def dump_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    dump_content(response, destination)


In [2]:
id = "17J9dfyiA8jxtZ_y4Yl7vZ2RenBK73AzC"
destination = "lvmdrp_src.zip"

In [3]:
download_file_from_google_drive(id, destination)